### Download HRRR data on public archives
Wei-Ming Tsai <br> Date modified: July, 2021 <br>
 - (2017-now, data will be gradually reduced)HRRR data archive: <br> http://home.chpc.utah.edu/~u0553130/Brian_Blaylock/hrrr_script_tips.html
 - (2014-now) Google Cloud Platform: <br>
 https://console.cloud.google.com/marketplace/product/noaa-public/hrrr?project=python-232920&pli=1

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
#import cfgrib
import cf2cdm
from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
from datetime import datetime, timedelta
import urllib.request
#from cfgrib.xarray_store import open_dataset
import warnings
import h5py

In [2]:
warnings.filterwarnings('ignore')

In [3]:
def datetime_converter(year, day_num, hour):

# adjusting day num in a certain year into year, month, day, hour
# day_num = string '300'
# year = string : '2016'
# hour = string, example: '02'

    day_num.rjust(3 + len(day_num), '0')

    # Initializing start date
    strt_date = datetime(int(year), 1, 1)

    # converting to date
    res_date = strt_date + timedelta(days=int(day_num) - 1)
    res_time = res_date + timedelta(hours=int(hour))
    
    return res_time

In [4]:
datetime_converter('2017','280','02')

datetime.datetime(2017, 10, 7, 2, 0)

In [5]:
os.chdir('/data2/willytsai/InSAR_HRRR/')
file = xr.open_dataset('date_ARs_events.nc')
date_acqui = []
for t in range(len(file.time)):
    tmp = datetime.utcfromtimestamp(file.time[t].values.tolist()/1e9)
    time_str = tmp.strftime('%Y%m%d%H')
    date_acqui.append(time_str) # times of acquisitions

In [6]:
date_acqui

['2014020821',
 '2014022621',
 '2014022700',
 '2014022703',
 '2014022706',
 '2014022806',
 '2014022809',
 '2014022812',
 '2014022818',
 '2014022821',
 '2014030100',
 '2014030103',
 '2014030106',
 '2014030109',
 '2014030112',
 '2014030115',
 '2014091800',
 '2014091803',
 '2014091806',
 '2014091809',
 '2014091812',
 '2014111309',
 '2014120215',
 '2014120218',
 '2014120221',
 '2014120300',
 '2014120303',
 '2014120306',
 '2014120309',
 '2014120312',
 '2014120315',
 '2014120318',
 '2014120521',
 '2014121109',
 '2014121112',
 '2014121115',
 '2014121118',
 '2014122103',
 '2014122106',
 '2014122109',
 '2015020612',
 '2015020615',
 '2015020618',
 '2015020621',
 '2015020700',
 '2015020703',
 '2015020706',
 '2015020709',
 '2015020812',
 '2015020815',
 '2015020818',
 '2015020821',
 '2015020900',
 '2015020903',
 '2015071909',
 '2015071912',
 '2015071915',
 '2015071918',
 '2015071921',
 '2015122121',
 '2015122200',
 '2015122203',
 '2015122206',
 '2015122209',
 '2015122212',
 '2015122215',
 '20151222

### Download HRRR data from archive
1. archive at Utah is reduced....
2. now using Google cloud platform. Fancy!

In [ ]:
os.system('mkdir -p /data2/willytsai/InSAR_HRRR/HRRR_ARs')
os.chdir('/data2/willytsai/InSAR_HRRR/HRRR_ARs')
# Downloading best analyzed output for specfied UTC.
# example: 
for date in date_acqui:
    date_str = date[:-2]
    hour_str = date[-2:]
    file_name = 'hrrr.'+date_str+'.t'+hour_str+'z.grib2'
    url = 'curl -o ' + file_name + ' '\
      'https://storage.googleapis.com/high-resolution-rapid-refresh/hrrr.'+date_str+'/conus/hrrr.t'+hour_str+'z.wrfprsf00.grib2'
    os.system(url)
    print(file_name)

### Extract precipitable water and merge into a single ncfile

In [ ]:
# get geolocation from InSAR
geo_file = '/data2/willytsai/InSAR_HRRR/CentralNevadaSenAT166/mintpy/inputs/geometryRadar.h5'
geo = h5py.File(geo_file,'r')
# for key in geo.keys():
#     print(key) #Names of the groups in HDF5 file.
lat = geo['latitude'];
lon = geo['longitude'];
incidence = geo['incidenceAngle'];
axis_bound = [np.min(lat),np.max(lat),np.min(lon),np.max(lon)]; # coordinate bound [South,North,West,East]
axis_bound = [np.unique(lat.value)[1],np.unique(lat.value)[-1],np.unique(lon.value)[0],np.unique(lon.value)[-2]]
axis_bound

In [ ]:
# using fake axis bound for matching ARs
axis_bound = [30,40,-124,-116]
axis_bound

In [ ]:
# merge HRRR dataset 
os.chdir('/data2/willytsai/InSAR_HRRR/HRRR_ARs/regrid_3km/')
files = sorted(glob('*grib2'))
tmp = xr.open_dataset(files[-1],engine='cfgrib',
                            backend_kwargs=dict(filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'unknown'}))
pwat_tmp = tmp.pwat.sel(latitude=slice(axis_bound[0],axis_bound[1]),longitude=slice(axis_bound[2]+360,axis_bound[3]+360))

pwat_acqu = np.zeros((len(files),pwat_tmp.shape[0],pwat_tmp.shape[1]))
date_frame = []

for t in range(len(files)):
    
    date_frame.append(datetime.strptime(files[t][5:18],'%Y%m%d.t%Hz'))
    try:
        ds = xr.open_dataset(files[t],engine='cfgrib',
                            backend_kwargs=dict(filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'unknown'}))
        pwat = ds.pwat.sel(latitude=slice(axis_bound[0],axis_bound[1]),longitude=slice(axis_bound[2]+360,axis_bound[3]+360))
        pwat_acqu[t,:,:] = pwat.values   
    except:
        print('ERROR file: '+files[t])
        pwat_acqu[t,:,:] = np.nan

# convert into xarray 
pwat_acqu_xr = xr.DataArray(pwat_acqu,dims=('time','latitude','longitude')
                            ,coords=(date_frame,pwat.latitude,pwat.longitude-360),name='pwat')
#pwat_acqu_xr.to_netcdf('/data2/willytsai/InSAR_HRRR/HRRR_data/t02z/merged/HRRR_pwat_NEVADA.nc')
os.system('mkdir -p merged') # create directory
pwat_ds = pwat_acqu_xr.to_dataset(name='pwat')
pwat_ds.to_netcdf('/data2/willytsai/InSAR_HRRR/HRRR_ARs/regrid_3km/merged/HRRR_pwat_ARs.nc')

### Checking merged files

In [ ]:
ds = xr.open_dataset('/data2/willytsai/InSAR_HRRR/HRRR_ARs/regrid_3km/merged/HRRR_pwat_ARs.nc')
ds

In [ ]:
fig = plt.figure(figsize=(10,4))

ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0))
states_provinces = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lakes_shp',
    scale='50m',
    facecolor='none')

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(states_provinces, edgecolor='gray')

plt.contourf(ds.longitude,ds.latitude,ds.pwat[-1,:,:])
plt.title(str(ds.time.values[-1])[:-10],fontsize=13)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='gray', alpha=0.5, linestyle=':')
gl.xlabels_top=False; gl.ylabels_right=False